In [ ]:
import pandas as pd
import math
import numpy as np
# from seqeval.metrics import f1_score
# from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import json
import random
import logging
import re
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# from pytorch_pretrained_bert import BertTokenizer, BertConfig
# from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

The dataset has 220 items of which 220 items have been manually labeled.

The labels are divided into following 10 categories:
*   Name
*   College Name
*   Degree
*   Graduation Year
*   Years of Experience
*   Companies worked at
*   Designation
*   Skills
*   Location
*   Email Address

In [ ]:
import pandas as pd
df = pd.read_json ('Entity Recognition in Resumes.json',lines=True)

In [ ]:
df.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [ ]:
for i in range(len(df)):
  df["content"][i] = df["content"][i].replace("\n", " ")
df.head()

<ipython-input-4-37e18527ab2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["content"][i] = df["content"][i].replace("\n", " ")


,content,annotation,extras
0,Abhishek Jha Application Development Associate...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar Active member of IIIT Committee...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina Hyderabad, Telangana - E...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai Operational Analyst (SQL DBA) Eng...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan lecturer - oracle tutorials Mum...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [ ]:
df = df.drop(['extras'], axis=1)
df.head()

,content,annotation
0,Abhishek Jha Application Development Associate...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar Active member of IIIT Committee...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina Hyderabad, Telangana - E...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai Operational Analyst (SQL DBA) Eng...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan lecturer - oracle tutorials Mum...,"[{'label': ['Degree'], 'points': [{'start': 20..."


In [ ]:
# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
  try:
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
      lines = f.readlines()

    for line in lines:
      data = json.loads(line)
      text = data['content'].replace("\n", " ")
      entities = []
      data_annotations = data['annotation']
      if data_annotations is not None:
        for annotation in data_annotations:
          # only a single point in text annotation.
          point = annotation['points'][0]
          labels = annotation['label']
          # handle both list of labels or a single label.
          if not isinstance(labels, list):
            labels = [labels]

          for label in labels:
            point_start = point['start']
            point_end = point['end']
            point_text = point['text']
                        
            lstrip_diff = len(point_text) - len(point_text.lstrip())
            rstrip_diff = len(point_text) - len(point_text.rstrip())
            if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
            if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
            entities.append((point_start, point_end + 1 , label))
      training_data.append((text, {"entities" : entities}))
    return training_data
  except Exception as e:
    logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
    return None 

In [ ]:
def trim_entity_spans(data: list) -> list:
  """Removes leading and trailing white spaces from entity spans.
  Args:
    data (list): The data to be cleaned in spaCy JSON format.

  Returns:
    list: The cleaned data.
  """
  invalid_span_tokens = re.compile(r'\s')

  cleaned_data = []
  for text, annotations in data:
    entities = annotations['entities']
    valid_entities = []
    for start, end, label in entities:
      valid_start = start
      valid_end = end
      while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
        valid_start += 1
      while valid_end > 1 and invalid_span_tokens.match(text[valid_end - 1]):
        valid_end -= 1
      valid_entities.append([valid_start, valid_end, label])
    cleaned_data.append([text, {'entities': valid_entities}])
  return cleaned_data

In [ ]:
with open('Entity Recognition in Resumes.json', 'r') as f:
      lines = f.readlines()

In [ ]:
data = trim_entity_spans(convert_dataturks_to_spacy('Entity Recognition in Resumes.json'))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

In [ ]:
def clean_entities(training_data):
  clean_data = []
  for text, annotation in training_data:
        
    entities = annotation.get('entities')
    entities_copy = entities.copy()
        
    # append entity only if it is longer than its overlapping entity
    i = 0
    for entity in entities_copy:
      j = 0
      for overlapping_entity in entities_copy:
        # Skip self
        if i != j:
          e_start, e_end, oe_start, oe_end = entity[0], entity[1], overlapping_entity[0], overlapping_entity[1]
          # Delete any entity that overlaps, keep if longer
          if ((e_start >= oe_start and e_start <= oe_end) \
          or (e_end <= oe_end and e_end >= oe_start)) \
          and ((e_end - e_start) <= (oe_end - oe_start)):
            entities.remove(entity)
        j += 1
      i += 1
    clean_data.append((text, {'entities': entities}))
                
  return clean_data

In [ ]:
data = clean_entities(data)

In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import math

def train_test_split(data, test_size, random_state):
  random.Random(random_state).shuffle(data)
  test_idx = len(data) - math.floor(test_size * len(data))
  train_set = data[0: test_idx]
  test_set = data[test_idx: ]
  return train_set, test_set

In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
from spacy.training.example import Example

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import spacy

def train_spacy():  
  nlp = spacy.blank('en')  # create blank Language class
  # create the built-in pipeline components and add them to the pipeline
  # nlp.create_pipe works for built-ins that are registered with spaCy
  if 'ner' not in nlp.pipe_names:
    # ner = nlp.create_pipe('ner')
    ner = nlp.add_pipe("ner", last=True)
        
  # add labels
  for _, annotations in train_data:
    for ent in annotations.get("entities"):
      ner.add_label(ent[2])
            
  # get names of other pipes to disable them during training
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(10):
      print("Statring iteration " + str(itn))
      random.shuffle(train_data)
      losses = {}
      for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # Update the model
        nlp.update([example], losses=losses, drop=0.3, sgd=optimizer)
      print(losses)
      print("----------------------------")
  return nlp

In [ ]:
nlp = train_spacy()

Statring iteration 0


/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Jatin Arora SDET Automation Engineer, Infosys - CR..." with entities "[[3909, 3931, 'College Name'], [3883, 3907, 'Degre...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Laya A Cluster HR Manager - Velammal New  Chennai,..." with entities "[[3760, 4638, 'Skills'], [3727, 3742, 'College Nam...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be ali

{'ner': 12760.41460258184}
----------------------------
Statring iteration 1
{'ner': 7302.462215911474}
----------------------------
Statring iteration 2
{'ner': 4255.182554326091}
----------------------------
Statring iteration 3
{'ner': 3945.9499197825758}
----------------------------
Statring iteration 4
{'ner': 3333.0367380629336}
----------------------------
Statring iteration 5
{'ner': 3201.461978673648}
----------------------------
Statring iteration 6
{'ner': 2980.876323831962}
----------------------------
Statring iteration 7
{'ner': 2757.4431122900064}
----------------------------
Statring iteration 8
{'ner': 2697.9443489947957}
----------------------------
Statring iteration 9
{'ner': 2575.0731660995825}
----------------------------


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=06abe3105c9ab5e09e57443bd71eb80fe5bea5f66b9e4476c57c2afeb54e4a82
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
# from spacy.gold import GoldParse
from spacy.training import offsets_to_biluo_tags
from itertools import groupby

In [ ]:
def doc_to_bilou(nlp, text):
  doc = nlp(text)
  tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
  entities = []
  for entity, group in groupby(tokens, key=lambda t: t[-1]):
    if not entity:
      continue
    group = list(group)
    _, start, _ = group[0]
    word, last, _ = group[-1]
    end = last + len(word)
        
    entities.append((start, end, entity))

  gold = offsets_to_biluo_tags(nlp(text), entities = entities)
  pred_ents = gold
    
  return pred_ents

In [ ]:
for text, annots in test_data:
  doc = nlp(text)
  for e in doc:
    print(e.text, e.ent_iob_, e.ent_type_)
  # print([(e.text, e.ent_iob_, e.ent_type_) for e in doc])
  break

Soumya B Name
Balan I Name
Soumya B Location
Balan I Location
- O 
BE O 
Computer O 
Science O 
- O 
3 O 
yr O 
Work O 
Experience O 
at O 
Microsoft O 
Corporation O 
  O 
Thiruvananthapuram O 
, O 
Kerala O 
- O 
Email O 
me O 
on O 
Indeed O 
: O 
indeed.com/r/Soumya- B Email Address
Balan/8c7fbb9917935f20 I Email Address
  O 
➢ O 
To O 
work O 
in O 
a O 
progressive O 
organization O 
where O 
I O 
can O 
enhance O 
my O 
skills O 
and O 
learning O 
to O 
contribute O 
to O 
the O 
success O 
of O 
the O 
organization O 
. O 
  O 
Willing O 
to O 
relocate O 
: O 
Anywhere O 
  O 
WORK O 
EXPERIENCE O 
  O 
Technical B Designation
Support I Designation
Engineer I Designation
  O 
Microsoft B Companies worked at
iGTSC O 
- O 
  O 
Bengaluru B Location
, O 
Karnataka O 
- O 
  O 
July O 
2013 O 
to O 
October O 
2015 O 
  O 
Position O 
: O 
TECHNICAL O 
SUPPORT O 
ENGINEER O 
  O 
Company O 
: O 
Microsoft O 
Corporation O 
- O 
Microsoft B Companies worked at
India O 
Global O 
T

In [ ]:
# from spacy import displacy

# for text, annots in test_data:
#   doc = nlp(text)
#   tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
#   entities = []
#   for entity, group in groupby(tokens, key=lambda t: t[-1]):
#     if not entity:
#       continue
#     group = list(group)
#     _, start, _ = group[0]
#     word, last, _ = group[-1]
#     end = last + len(word)
        
#     entities.append((start, end, entity))
#   displacy.render(doc, style = "ent", jupyter = True)
#   break

In [ ]:
y_test = []
y_pred = []

for text, annots in test_data:
  gold = offsets_to_biluo_tags(nlp.make_doc(text), entities = annots.get("entities"))
  ents = gold
  pred_ents = doc_to_bilou(nlp, text)
    
  y_test.append(ents)
  y_pred.append(pred_ents)

/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Chaban kumar Debbarma Tripura - Email me on Indeed..." with entities "[[277, 328, 'Email Address'], [257, 263, 'College ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Nitin Verma Assisting Microsoft Partners - Exchang..." with entities "[[1308, 1349, 'Email Address'], [1277, 1281, 'Loca...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be ali

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from itertools import chain

In [ ]:
from seqeval.metrics import f1_score

In [ ]:
lb = LabelBinarizer()
y_true_combined = lb.fit_transform(list(chain.from_iterable(y_test)))
y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

In [ ]:
def ner_report(y_true, y_pred):
  """
  Classification report for a list of BIO-encoded sequences.
  It computes token-level metrics and discards "O" labels.
    
  Note that it requires scikit-learn 0.15+ (or a version from github master)
  to calculate averages properly!
  """
  lb = LabelBinarizer()
  y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
  y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
  tagset = set(lb.classes_)
  tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
  class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
  return classification_report(
    y_true_combined,
    y_pred_combined,
    labels = [class_indices[cls] for cls in tagset],
    target_names = tagset
    ), accuracy_score(y_true_combined, y_true_combined
    ), f1_score(y_test, y_pred)

In [ ]:
report, accuracy, score = ner_report(y_test, y_pred)
print(report)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-Name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-Degree seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-Years of Experience seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-Companies worked at s

                       precision    recall  f1-score   support

                    -       0.00      0.00      0.00       428
       B-College Name       0.60      0.59      0.60        54
       I-College Name       0.63      0.51      0.57       103
       L-College Name       0.53      0.52      0.52        54
       U-College Name       0.00      0.00      0.00         6
B-Companies worked at       0.69      0.42      0.52        48
I-Companies worked at       0.00      0.00      0.00        10
L-Companies worked at       0.66      0.40      0.49        48
U-Companies worked at       0.37      0.29      0.33        92
             B-Degree       0.89      0.56      0.68        45
             I-Degree       0.82      0.59      0.69       132
             L-Degree       0.86      0.53      0.66        45
             U-Degree       0.38      0.60      0.46         5
        B-Designation       0.65      0.43      0.52        91
        I-Designation       0.64      0.33      0.43  

In [ ]:
print(accuracy)

1.0


In [ ]:
print(score)

0.5014299332697807
